In [1]:
from jettool import tools
import pandas
import numpy

# 初始化

In [2]:
# 初始化物件，可同時set Key
tejtool = tools.financial_tool(api_key="GDEy0mWAGqnI3EemCREGREZMcEVbnF")
# tejtool.load_data()
# 設定keyID，會自動查詢需要的資料表索引構造
#tejtool.set_apikey("GDEy0mWAGqnI3EemCREGREZMcEVbnF")

['使用者名稱：趙育祥(ZYX)', '使用權限日期：2018-01-01/2050-12-31', '日連線次數狀態：575/100000', '日查詢資料量狀態：4166024/10000000', '月查 詢資料量狀態：37666689/9223372036854775807']


# 資料結構資訊

In [ ]:
# 尋找某個table的mapping資料次屬性，可以用來知道你查的table，在其他國家是什麼table
tejtool.get_table_mapping(market='TWN',id='AIND')

In [ ]:
# 取得該table除了PK以外的欄位名稱，用來簡化查詢步驟，無腦一口氣整個表匯入
tejtool.get_table_columns('TWN/AFF_RAW')

In [ ]:
# 用模糊字串來查詢確切的會計科目名稱
tejtool.find_account_name('資產負債')

# 整合查詢器
### 一次查詢全部整合完成
#### 一個名稱、可到不同資料表查詢相對應的資料
#### 需正確定義名稱的精確差異

In [4]:
#data_name = ['常續性稅後淨利','收盤價(元)','報酬率-Ln','個別風險','長期反轉(T-60~T-13)','董監持股%','大股東持股(TSE)%','經理人持股%','單月營收(千元)']
fama_name = tejtool.get_table_columns('TWN/AFF_RAW')
data_name = ['常續性稅後淨利','收盤價(元)','報酬率-Ln','單月營收(千元)']
data_name = data_name + fama_name
data = tejtool.query_data(base_date='2019-12-31',window='36m',column_names=data_name)
tejtool.save_data()
data[data['coid']=='2330']

資料起始日：2016-12-31
查詢財報資料
成功查詢會計家數:2036
最大財報資料日期:2019-09-01
查詢日資料 最大資料日期:2019-12-31
借券餘額 重新查詢:False
上市(櫃)調整股價(日)-除權息調整 重新查詢:False
台灣_多因子DB 重新查詢:False
上市(櫃)月營收盈餘 重新查詢:False


,zdate,mdate,coid,違約距離(KMV),研發密度,短期反轉(T-1),盈利能力(ROE),流動性,市值,動能(T-12~T-2),...,長期反轉(T-60~T-13),錯誤定價,單月營收(千元),違約距離(MERTON),個別風險,股利殖利率,常續性稅後淨利,產業集中度,收盤價(元),投資(資產成長率)
427670,2017-01-04,2016-12-01,2330,0.0,3.4251,-5.7471,-9.48,1.8458,7.898290e+06,0.0,...,-40.6780,-43.1924,1299155.0,0.0,1.9463,0.0,0.0,0.26,183.0,0.21
427671,2017-01-05,2016-12-01,2330,0.0,3.4251,-5.7471,-9.48,1.8458,7.898290e+06,0.0,...,-40.6780,-43.1924,1299155.0,0.0,1.9463,0.0,0.0,0.26,183.5,0.21
427672,2017-01-06,2016-12-01,2330,0.0,3.4251,-5.7471,-9.48,1.8458,7.898290e+06,0.0,...,-40.6780,-43.1924,1299155.0,0.0,1.9463,0.0,0.0,0.26,184.0,0.21
427673,2017-01-09,2016-12-01,2330,0.0,3.4251,-5.7471,-9.48,1.8458,7.898290e+06,0.0,...,-40.6780,-43.1924,1299155.0,0.0,1.9463,0.0,0.0,0.26,184.0,0.21
427674,2017-01-10,2016-12-01,2330,0.0,3.4251,-5.7471,-9.48,1.8458,7.898290e+06,0.0,...,-40.6780,-43.1924,78112022.0,0.0,1.9463,0.0,0.0,0.26,184.0,0.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428399,2019-12-25,2019-09-01,2330,0.0,1.3860,9.7425,24.99,0.0094,6.197361e+09,0.0,...,105.8228,-91.6598,107884396.0,0.0,0.6291,0.0,101717149.0,1189.19,333.0,4.93
428400,2019-12-26,2019-09-01,2330,0.0,1.3860,9.7425,24.99,0.0094,6.197361e+09,0.0,...,105.8228,-91.6598,107884396.0,0.0,0.6291,0.0,101717149.0,1189.19,333.0,4.93
428401,2019-12-27,2019-09-01,2330,0.0,1.3860,9.7425,24.99,0.0094,6.197361e+09,0.0,...,105.8228,-91.6598,107884396.0,0.0,0.6291,0.0,101717149.0,1189.19,338.0,4.93
428402,2019-12-30,2019-09-01,2330,0.0,1.3860,9.7425,24.99,0.0094,6.197361e+09,0.0,...,105.8228,-91.6598,107884396.0,0.0,0.6291,0.0,101717149.0,1189.19,334.5,4.93


# 回測工具，與quantjet網站同一套

In [ ]:
# 定義計算式，然後使用back_test函式即可回測，目前欠缺自動查詢函式內需要資料欄位(即quantjet網站js檔的功能)的程序

def calculate():
    tejtool.data['EP'] = tejtool.data['常續性稅後淨利']/tejtool.data['收盤價(元)']
    tejtool.data['常續性稅後淨利檢查'] = tejtool.check_above('EP',0,window='1d')
def evaluate():
    tejtool.data['購入'] = tejtool.check_condition(conditions=['常續性稅後淨利檢查'])
    tejtool.data['unit'] = tejtool.equal_pv()

# 另外必須輸入用來當作市價與報酬率的欄位名稱，如:roib_name='報酬率-Ln',closed_name ='收盤價(元)'
# 若不輸入，則使用此預設值
tejtool.back_test(back_interval='2019-10-31',cash=1000000000,calculate=calculate,evaluate=evaluate)

In [5]:
def calculate(tejtool):
    tejtool.data['報酬率']=tejtool.calculate_growthrate(check_index='收盤價(元)',window='1m',fix_date='01',method='geometric')
    tejtool.data['famamodel']=tejtool.make_famamacbethmodel(col_name='報酬率-Ln',check_index=['市值', '淨值市價比', '益本比', '股利殖利率', '動能(T-12~T-2)', '短期反轉(T-1)', '長期反轉(T-60~T-13)', '盈利能力(ROE)', '投資(資產成長率)', '違約距離(MERTON)', '違約距離(KMV)', '流動性', '個別風險', '產業集中度', '錯誤定價', '偏態係數', '研發密度'],window ='24m',alpha_rate = 5,reset_list = '01')
    tejtool.data['famascore']=tejtool.run_famascore(rank_above=100,class_count = 10) 
    tejtool.data['fama分組當月報酬']=tejtool.calculate_crossing(check_index='報酬率',window='1d',weight='市值',col_kind='mean')
def evaluate(tejtool):
    tejtool.data['fama分組判斷'] =tejtool.abnormal_selection(check_index='fama分組當月報酬',group_name='famascore',window='12m',keep='first')
    tejtool.data['購入']=tejtool.check_condition(conditions=['fama分組判斷'],check_type='and') 
    tejtool.data['unit'] = tejtool.equal_pv()
tejtool.back_test(back_interval='2018-12-31',cash=1000000000,calculate=calculate,evaluate=evaluate)

跨股模型計算與回顧測試
[1000000000, 1000000000]
倒推計算日2018-12-28
回測損益起算日2018-12-31
back interval:
[242, -1]
開始計算指標


C:\Users\zyx\Documents\GitHub\jettool\pipeline\method.py:460: RuntimeWarning: divide by zero encountered in true_divide
  this_date_data['temp_mid'] = [0] + numpy.log(roib_y1/roib_y0).tolist()
C:\Users\zyx\Documents\GitHub\jettool\pipeline\method.py:460: RuntimeWarning: invalid value encountered in true_divide
  this_date_data['temp_mid'] = [0] + numpy.log(roib_y1/roib_y0).tolist()
C:\Users\zyx\Documents\GitHub\jettool\pipeline\method.py:460: RuntimeWarning: divide by zero encountered in log
  this_date_data['temp_mid'] = [0] + numpy.log(roib_y1/roib_y0).tolist()


2018-12-28


C:\Users\zyx\Documents\GitHub\jettool\pipeline\method.py:351: RuntimeWarning: divide by zero encountered in true_divide
  this_hold_units = numpy.floor(unit_pv/this_closed)


2019-01-02 持股現值:994846614
2019-01-03 持股現值:988701450
2019-01-04 持股現值:1001844911
2019-01-07 持股現值:1002650032
2019-01-08 持股現值:1010319888
2019-01-09 持股現值:1008732180
2019-01-10 持股現值:1006913500
2019-01-11 持股現值:1004217740


C:\Users\zyx\Documents\GitHub\jettool\pipeline\method.py:342: RuntimeWarning: divide by zero encountered in true_divide
  this_hold_units = numpy.floor(unit_pv/this_closed)


2019-01-14 持股現值:1009690899
2019-01-15 持股現值:1011763334
2019-01-16 持股現值:1011711426
2019-01-17 持股現值:1017550511
2019-01-18 持股現值:1020823801
2019-01-21 持股現值:1017879863
2019-01-22 持股現值:1019833627
2019-01-23 持股現值:1022911371
2019-01-24 持股現值:1027467365
2019-01-25 持股現值:1030697460
2019-01-28 持股現值:1027995682
2019-01-29 持股現值:1030323790
2019-01-30 持股現值:1034874022
2019-02-11 持股現值:1042878626
2019-02-12 持股現值:1046509732
2019-02-13 持股現值:1051216681
2019-02-14 持股現值:1048827207
2019-02-15 持股現值:1055889864
2019-02-18 持股現值:1060080818
2019-02-19 持股現值:1067011258
2019-02-20 持股現值:1069423715
2019-02-21 持股現值:1068627534
2019-02-22 持股現值:1074979749
2019-02-25 持股現值:1073619656
2019-02-26 持股現值:1073341535
2019-02-27 持股現值:1077545620
2019-03-04 持股現值:1075591287
2019-03-05 持股現值:1079452353
2019-03-06 持股現值:1070996794
2019-03-07 持股現值:1071052004
2019-03-08 持股現值:1075108004
2019-03-11 持股現值:1078609306
2019-03-12 持股現值:1080309280
2019-03-13 持股現值:1081185583
2019-03-14 持股現值:1084634085
2019-03-15 持股現值:1089354282
2019-03-18 持股現值:1089333095
2

C:\Users\zyx\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\plotting\_matplotlib\converter.py:103: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


total cost823.0360000133514


In [ ]:
tejtool.data.loc[tejtool.data['coid']=='1603',['unit','roibNext','前期持股','收盤價(元)','報酬率-Ln','手續費']]

In [ ]:
tejtool.all_date_data[(tejtool.all_date_data['coid']=='1603')&(tejtool.all_date_data['zdate']>='2019-08-15')]